In [4]:
from selenium_profiles.webdriver import Chrome
from selenium_profiles.profiles import profiles
from selenium.webdriver.common.by import By
from seleniumwire import webdriver
import time
import pandas as pd

In [10]:
profile = profiles.Windows() # or .Android
options = webdriver.ChromeOptions()
# profile['options']["extension_paths"] = ["C:/Users/Elmasry/AppData/Local/Google/Chrome/User Data/Default/Extensions/ejddcgojdblidajhngkogefpkknnebdh/1.9.8.1_0"]
profile['options']["load_images"] = False
# options.add_argument("--headless=new")
driver = Chrome(profile, options=options,
                uc_driver=False
                )
driver.get("https://store.steampowered.com/search/?filter=topsellers&category1=998&deck_compatibility=2&supportedlang=english&ndl=1")

#I'm going to start from 1 and end at 400
start = 800
end = 1200
#scrolling to the bottom of the page to load more games, each scroll loads ~25 games
for i in range(int(end/25)):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)


game_links = []
game_prices = {}
game_info = []
dev_info = []
links = []
genre = []
game_tags = []
i=0
try:
    games = driver.find_elements(By.XPATH, f'//*[@id="search_resultsRows"]/a[position()>{start} and position()<{end}]')
    if len(games) == 0: raise ValueError 
    for game in games:
        game_link = game.get_attribute('href')
        game_links.append(game_link)
        try:
            game_prices[game.find_element(By.XPATH, './div[2]/div[1]/span').text]= game.find_element(By.CLASS_NAME, 'discount_final_price').text
        except:
            game_prices[game.find_element(By.XPATH, './div[2]/div[1]/span').text]= None


    for link in game_links:
        i+=1
        driver.get(link)

        #if link contains 'sub' continue
        if 'sub' in link:
            print(f'Link: {link} is a sub, continuing...')
            continue

        #if link contains 'agecheck' pass it.
        if 'agecheck' in driver.current_url:
            try:
                # game.find_element(By.XPATH, '//*[@id="app_agegate"]/div[1]/div[2]/div').get_attribute('href')
                time.sleep(3)
                day = driver.find_element(By.ID, 'ageDay')
                day.click()
                day_menu = driver.find_element(By.XPATH, "//option[@value='4']")
                day_menu.click()
                month = driver.find_element(By.ID, 'ageMonth')
                month.click()
                month_menu = driver.find_element(By.XPATH, "//option[@value='April']")
                month_menu.click()
                year = driver.find_element(By.ID, 'ageYear')
                year.click()
                year_menu = driver.find_element(By.XPATH, "//option[@value='2000']")
                year_menu.click()
                view_page = driver.find_element(By.XPATH, '//*[@id="view_product_page_btn"]/span')
                view_page.click()
                print('Age gate passed, continuing...')
                time.sleep(3)

            except:
                print(f'Age gate failed on link: {link}, continuing...')
                pass
        driver.execute_script("""
        for(let script of document.querySelectorAll('script')) script.remove()
        """)
        try:
            game_name = driver.find_element(By.XPATH, '//*[@id="appHubAppName"]').text
            dev_info = driver.find_element(By.XPATH, '//*[@id="genresAndManufacturer"]/span')
            tags = driver.find_element(By.XPATH, '//*[@id="category_block"]/div[1]')
            genre = driver.find_element(By.XPATH, '//*[@id="genresAndManufacturer"]/span')
        except:
            print(f'Assigning lists failed on link: {link}\n continuing...')
            game_name = ""
            dev_info = ""
            tags = ""
            genre = ""
        try:
            price = game_prices[game_name]
        except:
            print(f'Price not found for game: {game_name}...')
            price = None
        try:
            Release_Date = driver.find_element(By.XPATH , '//*[@id="game_highlights"]/div[1]/div/div[3]/div[2]/div[2]').text
        except:
            try:
                Release_Date = driver.find_element(By.XPATH , '//*[@id="game_highlights"]/div[2]/div/div[3]/div[2]/div[2]').text
            except:
                print(f'Release date not found for game: {game_name}...')
                Release_Date = None
        try:
            number_of_reviews = driver.find_element(By.XPATH , '//*[@id="userReviews"]/div/div[2]/span[2]').text
        except:
            print(f'Number of reviews is different for game: {game_name}...')
            try:
                number_of_reviews = driver.find_element(By.XPATH , '//*[@id="userReviews"]/div/div[2]/span[1]')
            except:
                print(f'Number of reviews not found for game: {game_name}...')
                number_of_reviews = None
        try:
            publisher = driver.find_element(By.XPATH , '//*[@id="genresAndManufacturer"]/div[2]/a').text
        except:
            print(f'Publisher not found for game: {game_name}...')
            publisher = None
        try:
            game_info.append({
            "Name": game_name,
            "Review": driver.find_element(By.XPATH , '//*[@id="userReviews"]/div/div[2]/span[1]').text,
            "Number Of Reviews": number_of_reviews,
            "Genre": genre.text.split('\n'),
            "Tags": tags.text.split('\n'),
            "Dev_Info": dev_info.text.split('\n'),
            "Developer": driver.find_element(By.XPATH , '//*[@id="genresAndManufacturer"]/div[1]/a').text,
            "Publisher": publisher,
            "Release_Date": Release_Date,
            "link": link,
            "Price": game_prices[game_name]
            })
            print(f'{i}/{len(game_links)} Appending successful for game: {game_name}, continuing...')
            print('*'*50)
        except:
            print(f'Appending failed for game: {game_name}, continuing...')
            continue
            # print(driver.find_element(By.XPATH, '//*[@id="userReviews"]/div/div[2]/span[1]').text)
            # print(driver.find_element(By.XPATH, '//*[@id="userReviews"]/div/div[2]/meta[1]').get_attribute('content'))
            # print(genre.text.split('\n'))
            # print(tags.text.split('\n'))
            # print(dev_info.text.split('\n'))
            # print(driver.find_element(By.XPATH, '//*[@id="genresAndManufacturer"]/div[1]/a').text)
            # print(driver.find_element(By.XPATH, '//*[@id="genresAndManufacturer"]/div[2]/a').text)
            # print(driver.find_element(By.XPATH, '//*[@id="game_highlights"]/div[1]/div/div[3]/div[2]/div[2]').text)
            # print(link)
            # print(game_prices[game_name])


        links.append(link)

        
except:
    print("Couldn't fetch any more games...")
    df = pd.DataFrame(game_info)
    df.to_csv('steam_games.csv', index=False)
    driver.quit()

df = pd.DataFrame(game_info)
df.to_csv('Data/steam_games.csv', index=False)
print('Scraping finished successfully...')
print('All games selected: ',len(game_links))
print('Games successully scraped: ',len(game_info))
driver.quit()

1/399 Appending successful for game: Beasts of Bermuda, continuing...
**************************************************
2/399 Appending successful for game: MADNESS: Project Nexus, continuing...
**************************************************
3/399 Appending successful for game: Just Go, continuing...
**************************************************
4/399 Appending successful for game: Is It Wrong to Repay the Debt in a Dungeon?, continuing...
**************************************************
5/399 Appending successful for game: Drug Dealer Simulator, continuing...
**************************************************
6/399 Appending successful for game: Regiments, continuing...
**************************************************
7/399 Appending successful for game: AO Tennis 2, continuing...
**************************************************
8/399 Appending successful for game: Paragon: The Overprime, continuing...
**************************************************
Link: https://st